In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
import ptb_reader
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.models import load_model
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam

import keras.backend as K

import os

Using TensorFlow backend.


In [11]:
batch_size = 20
dataset_size = 16000
num_steps = 20
hidden_size_1 = 128
# hidden_size_2 = 64
feat_len = 10000

In [12]:
def seq_loss(y_true, y_pred):
    return (K.mean((K.categorical_crossentropy(y_pred, y_true)),axis=1))

In [13]:
def seq_acc(y_true, y_pred):
    cross_ent = seq_loss(y_true, y_pred)
    return K.pow(cross_ent,2)

In [14]:
def one_hot(x, y):
    xmat = np.zeros((x.shape[0], num_steps, feat_len))
    for j in range(x.shape[1]):
        for i in zip(range(num_steps), x[j]):
            xmat[j, i[0], i[1]] = 1
    
    ymat = np.zeros((x.shape[0], feat_len))
    for j in range(x.shape[0]):
        ymat[j, y[j]] = 1
    return xmat, ymat

In [15]:
def get_model():
    
    model = Sequential()
    
#     model.add(Embedding( output_dim=500, input_dim=feat_len, input_length=num_steps))
    model.add(Embedding( output_dim=500,input_dim=1, input_length=num_steps))
#     model.add(Embedding( num_steps, feat_len))
    # , dropout=0.2, recurrent_dropout=0.2
    model.add(LSTM(hidden_size_1, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
#     model.add(LSTM(hidden_size_2, return_sequences=True))
#     model.add(Dense(feat_len*num_steps, activation='softmax'))
    model.add(TimeDistributed(Dense(units = feat_len, input_dim = hidden_size_1, activation = 'softmax'))) 
    
    adam = Adam(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss=seq_loss,
              optimizer='adam',
              metrics=[ seq_acc])
    
    return model

In [ ]:
# model.add(Embedding(max_features, num_steps))
# model.add(LSTM(50, input_shape=(1,1), dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(vocab, activation='softmax'))
# model.pop()
# model.add(Dense(10, input_shape=(10,), activation='softmax'))

# model.pop()
# model.pop()





# plot_model(model, to_file='model.png')

In [16]:
def get_data(filepath):
    word_to_id = ptb_reader._build_vocab(filepath)
    return ptb_reader._file_to_word_ids(filepath, word_to_id), len(word_to_id)

In [17]:
def train(train_data, verbose = 0, model = None):
    if model is None:
        model = get_model()
    
    if(verbose > 0):
        print('Train...')

    for step, (x, y) in enumerate(ptb_reader.ptb_iterator(train_data, batch_size, num_steps)):    
#         x1 = np.zeros((dataset_size, num_steps, feat_len))
#         for i in range(x1.shape[0]):
#             for j in range(x1.shape[1]):
#                 x1[i,j,x[i,j]] = 1

#         y1 = y[:,-1]

#         y1 = np.zeros((dataset_size, feat_len))
#         for i in range(y1.shape[0]):
#             y1[i,y[i,-1]] = 1
#         print(x.shape)
#         print(y.shape)
#         x1, y1 = one_hot(x, y[:, -1])

#         print(x1.shape)
#         print(y1.shape)
        
#         x1_tf = np.reshape(x1, (batch_size, num_steps, feat_len))
        y1_tf = np.reshape(y, (batch_size, num_steps, 1))
        
        
        model.fit(x, y1_tf, epochs=10, verbose = verbose, batch_size=10)
        loss, perplexity  = model.evaluate(x, y1_tf, verbose = verbose, batch_size=10)

       
#         if(step % 100 == 0 and verbose> 0):
        if(step % 100 == 0 ):
            print('\n#',step+1, ' Perplexity: ', perplexity, 'CrossEnt Loss: ', loss)
        
        
    if( not os.path.isdir('weights') ):
        os.mkdir('weights')
        
    model.save('weights/my_model.h5')
    
#     model.save_weights('my_model_weights.h5')
    
    

In [9]:
def test(test_data, verbose = 0):

    model =load_model('weights/my_model.h5')

#     model = get_model()
#     model.load_weights('my_model_weights.h5')
    
    acc = 0.0
    siz = 0
    perplexity = []
    for step, (x, y) in enumerate(ptb_reader.ptb_iterator(test_data, 1000, num_steps)):
        
        x1, y1 = one_hot(x,y[:,-1])
        print(x1.shape)
        print(y1.shape)
        

        output = model.predict(x, verbose=verbose)
        score  = model.evaluate(x, y1, verbose = 1, batch_size=10)


        siz += 1 
        
        
        print('')
        print('Step: ',step+1, end='')
        print(', Test accuracy:', accuracy )
            
        
    print('Average Accuracy: ', acc/siz)
    
    return np.mean(perp_np)
    
    

In [ ]:
# if __name__ == "__main__":
#     global feat_len
#     FLAGS = tf.app.flags.FLAGS
#     tf.app.flags.DEFINE_string('test', '../data/ptb.test.txt',
#                        """Path to file for testing """)

# #     train_data, valid_data, test_data, vocab = ptb_reader.ptb_raw_data("../data")
#     train_data, feat_len = get_data('../data/ptb.train.txt')
#     model = None
#     model = train(train_data, verbose = 1, model = model)
    
#     test_data, _ = get_data(FLAGS.test)
#     perp =test(test_data)
#     print(perp)
    

In [ ]:
#     train_data, valid_data, test_data, vocab = ptb_reader.ptb_raw_data("../data")
train_data, feat_len = get_data('../data/ptb.train.txt')
model = None
output,y = train(train_data, verbose = 0, model = model)


# 1  Perplexity:  4.92719995743e+16 CrossEnt Loss:  121056472.0


In [31]:
print(output.shape)
print(y.shape)

print(y[1,:])
print(np.argmax(output[1,:,:], axis=1))


(20, 20, 10000)
(20, 20)
[  42  105 3213    6 1357   52   22  207   10   14   42    1 1266   36    6
 1357    9 1149    5 2872]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
test_data, _ = get_data('../data/ptb.test.txt')
logit, label =test(test_data)
# print(perp)

In [ ]:
np.argmax(logit, axis=1)

In [ ]:
a= np.asarray([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,1]])


In [ ]:
a[:,[2,3]]

In [ ]:
np.exp2(a)